In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from textblob import TextBlob
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
import re
df = pd.read_csv("/home/ajadhav/Nostalgic/YoutubeComments.csv",usecols=['Comments','Nostalgic'],encoding='latin-1')
df = df.dropna(subset=['Nostalgic'], how='all')
df['Comments'] = df['Comments'].str.replace('\d+', '')
df.head()

,Comments,Nostalgic
0,.One of the many reasons I keep coming back to...,Yes
1,.Best song ever made,No
2,.quite possibly the grooviest chick i've ever ...,No
3,.If only had a timemachine get the hell out th...,Yes
4,.I wish I could time machine my head between t...,Yes


In [3]:
lb = preprocessing.LabelBinarizer()
df['Nostalgic'] = lb.fit_transform(df.Nostalgic)

In [4]:
df.head()

,Comments,Nostalgic
0,.One of the many reasons I keep coming back to...,1
1,.Best song ever made,0
2,.quite possibly the grooviest chick i've ever ...,0
3,.If only had a timemachine get the hell out th...,1
4,.I wish I could time machine my head between t...,1


In [5]:
df['Comments'] = df['Comments'].apply(lambda x: " ".join(x.lower() for x in x.split()))

df['Comments'] = df['Comments'].str.replace('[^\w\s]','')

stop = stopwords.words('english')
df['Comments'] = df['Comments'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['Comments'] = df.Comments.str.replace('[^a-zA-Z]', ' ')



In [6]:
df.head()

,Comments,Nostalgic
0,one many reasons keep coming back video remind...,1
1,best song ever made,0
2,quite possibly grooviest chick ive ever seen e...,0
3,timemachine get hell era go back one grewup,1
4,wish could time machine head thighs id love date,1


In [7]:
df['Comments'] = df['Comments'].apply(lambda x: str(TextBlob(x).correct()))

In [8]:
df['Comments'] = df['Comments'].apply(word_tokenize)

In [9]:
df.head()

,Comments,Nostalgic
0,"[one, many, reasons, keep, coming, back, video...",1
1,"[best, song, ever, made]",0
2,"[quite, possibly, grooves, thick, give, ever, ...",0
3,"[timemachine, get, hell, era, go, back, one, g...",1
4,"[wish, could, time, machine, head, thighs, id,...",1


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df['Comments'], df['Nostalgic'], test_size=0.33, random_state=42)

In [11]:
v = TfidfVectorizer()
df['Comments']=[" ".join(review) for review in df['Comments'].values]
v.fit(df['Comments'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [12]:
X_train = X_train.apply(lambda x: ' '.join(x))

X_train = v.transform(X_train)

X_test = X_test.apply(lambda x: ' '.join(x))

X_test = v.transform(X_test)

In [13]:
clf = MultinomialNB().fit(X_train,y_train )
predicted = clf.predict(X_test)

In [14]:
print('Results of Validation Set')
print(classification_report(y_test, predicted))
print(precision_recall_fscore_support(y_test, predicted, average='weighted'))

Results of Validation Set
             precision    recall  f1-score   support

          0       0.60      0.10      0.17        31
          1       0.65      0.96      0.78        55

avg / total       0.63      0.65      0.56        86

(0.63474016652311227, 0.65116279069767447, 0.55853853169174639, None)


In [15]:
TestData = pd.read_csv("/home/ajadhav/TransferLearning/YoutubeComments2.csv",usecols=['Comments','Nostalgic'],encoding='latin-1')
TestData = TestData.dropna(subset=['Nostalgic'], how='all')

FileNotFoundError: File b'/home/ajadhav/TransferLearning/YoutubeComments2.csv' does not exist

In [ ]:
TestData['Nostalgic'] = lb.fit_transform(TestData.Nostalgic)
TestData['Comments'] = TestData['Comments'].apply(lambda x: " ".join(x.lower() for x in x.split()))
TestData['Comments'] = TestData['Comments'].str.replace('[^\w\s]','')
TestData['Comments'] = TestData['Comments'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
TestData['Comments'] = TestData.Comments.str.replace('[^a-zA-Z]', ' ')
TestData['Comments'] = TestData['Comments'].apply(lambda x: str(TextBlob(x).correct()))
TestData['Comments'] = TestData['Comments'].apply(word_tokenize)

In [ ]:
Test = TestData['Comments'].apply(lambda x: ' '.join(x))
Test = v.transform(Test)

In [ ]:
predictedTest = clf.predict(Test)
print('Resluts on Testing data')
print(classification_report(TestData['Nostalgic'], predictedTest))
print(precision_recall_fscore_support(TestData['Nostalgic'], predictedTest, average='weighted'))